### 1. Importing Overlay

In [1]:
from pynq import Overlay
import time

In [2]:
riscv = Overlay("Overlay/risc_v_design.bit")

In [3]:
riscv.ip_dict.keys()`

dict_keys(['clk', 'core0_IOIn_0', 'core0_IOOut_0', 'core1_IOIn_1', 'core1_IOOut_1', 'ctrlSigBus', 'datToHost', 'host_datToMem', 'host_memAdd', 'host_memRd', 'host_memWr', 'rst', 'selCore1', 'useIMBit31'])

In [6]:
zero_32  = '00000000000000000000000000000000'
zero_16  = '0000000000000000'
zero_8   = '00000000'

### 2. Signal Initialization

In [7]:
core0_IOIn_0 = riscv.core0_IOIn_0
core0_IOOut_0 = riscv.core0_IOOut_0
core1_IOIn_1 = riscv.core1_IOIn_1
core1_IOOut_1 = riscv.core1_IOOut_1
datToHost = riscv.datToHost
host_datToMem = riscv.host_datToMem
host_memAdd = riscv.host_memAdd
host_memRd = riscv.host_memRd
host_memWr = riscv.host_memWr
selCore1 = riscv.selCore1
clk = riscv.clk
rst = riscv.rst

In [9]:
# Checking output signals

host_datToMem.write(0, int("ABCD", 16))
if hex(host_datToMem.read()) == "0xabcd":
    print("host_datToMem working!")
else: print("host_datToMem not working!")
host_datToMem.write(0, int(zero_32, 2))

host_memAdd.write(0, int("ABCD", 16))
if hex(host_memAdd.read()) == "0xabcd":
    print("host_memAdd working!")
else: print("host_memAdd not working!")
host_memAdd.write(0, int(zero_32, 2))

host_memRd.write(0, 1)
if hex(host_memRd.read()) == "0x1":
    print("host_memRd working!")
else: print("host_memRd not working!")
host_memRd.write(0, 0)

host_memWr.write(0, 1)
if hex(host_memWr.read()) == "0x1":
    print("host_memWr working!")
else: print("host_memWr not working!")
host_memWr.write(0, 0)

selCore1.write(0, 1)
if hex(selCore1.read()) == "0x1":
    print("selCore1 working!")
else: print("selCore1 not working!")
selCore1.write(0, 0)

clk.write(0, 1)
if hex(clk.read()) == "0x1":
    print("clk working!")
else: print("clk not working!")
clk.write(0, 0)

host_datToMem working!
host_memAdd working!
host_memRd working!
host_memWr working!
selCore1 working!
clk working!


### 3. Register Addresses

|Register|Address|
|:-------|:------|
|P0ToP1_0 (31:0)|0002 0000 00|
|P1ToP0_0 (31:0)|0002 0008 00|
|Instruction Memory|0004 0000-3FFC 00|
|runAll|0005 0048 00|

In [10]:
IMAddr_31_16 = int("0004", 16)
RBAddr_31_16 = int("0006", 16)
runAllAddr = int("00050048", 16)
P0ToP1_0Addr = int("00020000", 16)
P1ToP0_0Addr = int("00020008", 16)
pcAddr = int("00045000", 16)

### 4. Functions

#### Load Assembly Program Function

In [11]:
def load_program(program, core_no):
    
    # Checking valid core
    if core_no == 0 or core_no == 1:
        
        print("Programming core " + str(core_no) + "...")
        
        clk.write(0, 1)
        
        # Selecting core
        selCore1.write(0, core_no)
        
        # Creating memory address with IM Address MSB
        host_memAdd_31_16 = IMAddr_31_16
        host_memAdd_1_0 = int("00", 2)
        
        # Setting Write Mode
        host_memRd.write(0, 0)
        host_memWr.write(0, 1)
        
        clk.write(0, 0)
        
        j = 0 
        
        for i in range(0, 64):
            
            clk.write(0, 1)
            
            # Completing LSB of IM Address using the loop variable
            host_memAdd_15_2 = i
            host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:014b}'.format(host_memAdd_15_2) + '{0:02b}'.format(host_memAdd_1_0)
            # Writing the address
            host_memAdd.write(0,int(host_memAdd_str, 2))
            
            # print(host_memAdd_str)
            
            # Converting the hex program value into binary
            val = '{0:032b}'.format(int(program[i], 16))
            
            # Writing the program to the memory
            host_datToMem.write(0, int(val, 2))
                        
            clk.write(0, 0)
        
        # Clearing all outputs
        clk.write(0, 1)
        host_memWr.write(0, 0)
        host_memAdd.write(0, int('00000000000', 2))
        host_datToMem.write(0, int(zero_32, 2))
        clk.write(0, 0)
        
    else: print("Invalid core number")


#### Read Instruction Memory function

In [12]:
def read_IM(core_no):
    
    # Checking valid core
    if core_no == 0 or core_no == 1:
        
        print("\nReading Core " + str(core_no) + " IM...\n" )
        
        data = []
        
        clk.write(0, 1)
        
        # Selecting core
        selCore1.write(0, core_no)
        host_datToMem.write(0, int(zero_32, 2))
        
        # Creating memory Address with IM Address MSB
        host_memAdd_31_16 = IMAddr_31_16
        host_memAdd_1_0 = int("00", 2)
        
        host_memWr.write(0, 0)
       
        clk.write(0, 0)
        
        for i in range(0, 64):
            
            clk.write(0, 1)
            
            # Completing the IM address with loop variable
            host_memAdd_15_2 = i
            host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:014b}'.format(host_memAdd_15_2) + '{0:02b}'.format(host_memAdd_1_0)
            host_memAdd.write(0,int(host_memAdd_str,2))
            
            # Asserting read
            host_memRd.write(0, 1)
            
            clk.write(0, 0)
            
            clk.write(0, 1)
                
            # Reading data from datToHost and adding to an array
            data.append(hex(datToHost.read()))
            
            clk.write(0, 0)
            
            # Deasse
            clk.write(0, 1)
            host_memRd.write(0, 0)
            clk.write(0, 0)
            
        
        clk.write(0, 1)
        host_memWr.write(0, 0)
        host_memRd.write(0, 0)
        host_memAdd.write(0, int(zero_32, 2))
        host_datToMem.write(0, int(zero_32, 2))
        clk.write(0, 0)
        
        print(data)
    
    else: print("Invalid core number")

#### Activate Processor Core Function

In [13]:
def activate_processor(core_no):
    
    if core_no == 0 or core_no == 1:
        
        clk.write(0, 1)
        
        # Selecting core
        selCore1.write(0, core_no)
        
        # Setting write mode
        host_memRd.write(0, 0)
        host_memWr.write(0, 1)
        
        # Writing the runAll Register address to host_memAdd
        host_memAdd.write(0, runAllAddr)
        clk.write(0, 0)
        clk.write(0, 1)
        # Asserting the runAll Register
        host_datToMem.write(0, 1)
        clk.write(0, 0)
        
        # Clearing all outputs
        clk.write(0, 1)
        host_memRd.write(0, 0)
        host_memWr.write(0, 0)
        host_memAdd.write(0, int(zero_32, 2))
        host_datToMem.write(0, int(zero_32, 2))
        clk.write(0, 0)
        
        print("Core " + str(core_no) + " Activated!" )
                
    else: print("Invalid core number")

#### Deactivate Processor Core Function

In [14]:
def deactivate_processor(core_no):
    
    if core_no == 0 or core_no == 1:
        
        clk.write(0, 1)
        
        # Selecting core
        selCore1.write(0, core_no)
        
        # Setting write mode
        host_memRd.write(0, 0)
        host_memWr.write(0, 1)
        
        # Writing the runAll Register address to host_memAdd
        host_memAdd.write(0, runAllAddr)
        clk.write(0, 0)
        clk.write(0, 1)
        # Deasserting the runAll Register
        host_datToMem.write(0, 0)
        clk.write(0, 0)
        
        # Clearing all Outputs
        clk.write(0, 1)
        host_memRd.write(0, 0)
        host_memWr.write(0, 0)
        host_memAdd.write(0, int(zero_32, 2))
        host_datToMem.write(0, int(zero_32, 2))
        clk.write(0, 0)
        
        print("Core " + str(core_no) + " Deactivated!" )
        
    else: print("Invalid core number")

#### Read Register Function

In [15]:
def read_register(register_addr, core_no):
    
    if core_no == 0 or core_no == 1:
        
        clk.write(0, 1)
        
        # Selecting core 
        selCore1.write(0, core_no)
        host_datToMem.write(0, int(zero_32, 2))
        
        # Creating memory address with RB Addres MSB and selected register address
        host_memAdd_31_16 = RBAddr_31_16
        host_memAdd_15_2 = int(register_addr, 16)
        host_memAdd_1_0 = int("00", 2)
        host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:014b}'.format(host_memAdd_15_2) + '{0:02b}'.format(host_memAdd_1_0)
        
        # Setting read mode
        host_memWr.write(0, 0)
        host_memRd.write(0, 1)
        
        # Writing the register address to host_memAdd
        host_memAdd.write(0,int(host_memAdd_str,2))
        # print(host_memAdd_str)
        clk.write(0, 0)
        
        # Reading the register value
        clk.write(0, 1)
        data = hex(datToHost.read())
        clk.write(0, 0)
        
        # Clearing outputs
        clk.write(0, 1)
        host_memRd.write(0, 0)
        clk.write(0, 0)
        
        # print("Value of x" + register_addr + " in core " + str(core_no) + " = " + str(data))
        
        return data
    else: print("Invalid core number")

#### Read all Register Function

In [16]:
def read_all_registers(core_no):
    
    if core_no == 0 or core_no == 1:
        
        data = []
        
        clk.write(0, 1)
        
        # Selecting Core
        selCore1.write(0, core_no)
        
        # Clearing output
        host_datToMem.write(0, int(zero_32, 2))
        
        # Addressing Registers
        host_memAdd_31_16 = RBAddr_31_16
        host_memAdd_1_0 = int("00", 2)
        
        host_memWr.write(0, 0)
        clk.write(0, 0)
        
        # Looping throug 32 register addresses
        for i in range(0, 32):
            
            # Completeing the register address using loop vairable
            host_memAdd_15_2 = i
            host_memAdd_str = '{0:016b}'.format(host_memAdd_31_16) + '{0:014b}'.format(host_memAdd_15_2) + '{0:02b}'.format(host_memAdd_1_0)
     
            # print(host_memAdd_str)
            
            clk.write(0, 1)
            # Writing the register address
            host_memAdd.write(0,int(host_memAdd_str,2))
            
            # Asserting memRead
            host_memRd.write(0, 1)
            clk.write(0, 0)
            
            # Reading register value and adding to array
            clk.write(0, 1)
            data.append(hex(datToHost.read()))
            clk.write(0, 0)
            
            clk.write(0, 1)
            # Deasserting memRead
            host_memRd.write(0, 0)
            clk.write(0, 0)
            
        #print("\nx0 to x31 registers value in core " + str(core_no) + ":\n")
        #print(data)
        
        return data
    
    else: print("Invalid core number")

#### Read data from the specified memory address

In [17]:
def read_mem(mem_add, core_no):
    
    clk.write(0, 1)
    
    # Selecting Core
    selCore1.write(0, core_no)
    
    # Setting read mode
    host_memRd.write(0, 1)
    host_memWr.write(0, 0)
    
    # Writing memory address
    host_memAdd.write(0, mem_add)
    
    clk.write(0, 0)
    
    # Reading data from the processor
    clk.write(0, 1)
    data = hex(datToHost.read())
    clk.write(0, 0)
    
    # Clearing outputs
    clk.write(0, 1)
    host_memRd.write(0, 0)
    host_memWr.write(0, 0)
    host_memAdd.write(0, int(zero_32, 2))
    host_datToMem.write(0, int(zero_32, 2))
    clk.write(0, 0)
    
    return data

#### Read P0ToP1_0 Signal Function

In [18]:
def read_P0ToP1_0(core_no):
    
    if core_no == 0 or core_no == 1:
        
        data = read_mem(P0ToP1_0Addr, core_no)
        
        # print("P0ToP1_0 data in core " + str(core_no) + " = " + str(data))
    
        return data
    
    else: print("Invalid core number")

#### Read P1ToP0_0 Signal Function

In [19]:
def read_P1ToP0_0(core_no):
    
    if core_no == 0 or core_no == 1:
        
        data = read_mem(P1ToP0_0Addr, core_no)
        
        # print("P1ToP0_0 data in core " + str(core_no) + " = " + str(data))
        
        return data
    
    else: print("Invalid core number")

#### Read program counter 

In [20]:
def read_pc(core_no):
    
    if core_no == 0 or core_no == 1:
     
        pc_val = read_mem(pcAddr, core_no)           
    
        print("PC value in Core " + str(core_no) + " = " + pc_val)
        
    else: print("Invalid core number")

#### Processor Reset Function 

In [21]:
def reset_sys(core_no):
    clk.write(0, 1)
    rst.write(0, 1)
    clk.write(0, 0)
    time.sleep(0.1)
    clk.write(0, 1)
    rst.write(0, 0)
    clk.write(0, 1)

### 5. Testing

In [22]:
clearIM = [
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000",
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000"  
]

memData64 = [
    "deadbeef", "f00dcafe", "5a5a5a5a", "a5a5a5a5", "96969696", "69696969", "3c3c3c3c", "c3c3c3c3",
   "beefdead", "cafef00d", "a5a5a5a5", "5a5a5a5a", "69696969", "96969696", "c3c3c3c3", "3c3c3c3c",
   "deadbeef", "f00dcafe", "5a5a5a5a", "a5a5a5a5", "96969696", "69696969", "3c3c3c3c", "c3c3c3c3",
   "beefdead", "cafef00d", "a5a5a5a5", "5a5a5a5a", "69696969", "96969696", "c3c3c3c3", "3c3c3c3c",
   "deadbeef", "f00dcafe", "5a5a5a5a", "a5a5a5a5", "96969696", "69696969", "3c3c3c3c", "c3c3c3c3",
   "beefdead", "cafef00d", "a5a5a5a5", "5a5a5a5a", "69696969", "96969696", "c3c3c3c3", "3c3c3c3c",
   "deadbeef", "f00dcafe", "5a5a5a5a", "a5a5a5a5", "96969696", "69696969", "3c3c3c3c", "c3c3c3c3",
   "beefdead", "cafef00d", "a5a5a5a5", "5a5a5a5a", "69696969", "96969696", "c3c3c3c3", "3c3c3c3c"]

Writing into x1, x2 and x3 Registers

```assembly
addi x1, x0, 0x1a
addi x2, x0, 0x2b
add  x3, x1, x2 
```

In [24]:
testProgram = [
  "01A00093", "02B00113", "002081B3", "00000000", "00000000", "00000000", "00000000", "00000000",
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000"  
]

In [25]:
load_program(testProgram, 0)

# Readback data from the IM
read_IM(0)

reset_sys(0)
activate_processor(0)

# Read all registers
print("\nRegister x0 to x31 values in core 0:")
read_all_registers(0)

Core 0 Deactivated!
Programming core 0...

Reading Core 0 IM...

['0x1a00093', '0x2b00113', '0x2081b3', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
Core 0 Activated!

Register x0 to x31 values in core 0:


['0x0',
 '0x1a',
 '0x2b',
 '0x45',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0']

### 6. Load Assembly program into Core 0

#### Assembly progam

```assembly
# Increment P0ToP1_0 every ~ second, repeating
addi  x13, x0, 0x2F              

lui  x2,  0x20000
srli x2,  x2, 12               # x2 = 0x00020000 process-processor port base address 
decrDelayCount:                # use long instruction loop to implement ~1 second delay
 addi  x13, x13,  -1           # decrement delayCount   
 bne   x13, x0, decrDelayCount # stay in loop if delayCount > 0 
addi  x13, x0, 0x2F             

addi x3, x3,  1		         # increment x3
sw   x3, 0(x2)                # store(write) x3 to 0x00020000(P0ToP1_0), core 1 to core 0
beq  x0,  x0, decrDelayCount  # unconditional branch
```

In [26]:
# Hex dump of the above assembly program
IMaddiProgram = [
  "02F00693", "20000137", "00C15113", "FFF68693", "FE069EE3", "02F00693", "00118193", "00312023",
  "FE0006E3", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000"]

In [27]:
load_program(IMaddiProgram, 0)
reset_sys(0)
activate_processor(0)

# Readback data from the IM
read_IM(0)

Programming core 0...
Core 0 Activated!

Reading Core 0 IM...

['0x2f00693', '0x20000137', '0xc15113', '0xfff68693', '0xfe069ee3', '0x2f00693', '0x118193', '0x312023', '0xfe0006e3', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']


In [32]:
read_all_registers(0)

['0x0',
 '0x0',
 '0x20000',
 '0x6',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x1f',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0',
 '0x0']

### 7. Application 1 - Interaction between core 0 and core 1

x3 in Core 0 is incremented every second and the updated value is sent to Core 1 which reads from the P1ToP0_0 port and writes the incomming data into x5

#### Core 0 Assembly program

```assembly
# Increment P0ToP1_0 every ~ second, repeating
addi  x13, x0, 0x2F              

lui  x2,  0x20000
srli x2,  x2, 12               # x2 = 0x00020000 process-processor port base address 
decrDelayCount:                # use long instruction loop to implement ~1 second delay
 addi  x13, x13,  -1           # decrement delayCount   
 bne   x13, x0, decrDelayCount # stay in loop if delayCount > 0 
addi  x13, x0, 0x2F             

addi x3, x3,  1                 # increment x3
sw   x3, 0(x2)                # store(write) x3 to 0x00020000(P0ToP1_0), core 1 to core 0
beq  x0,  x0, decrDelayCount  # unconditional branch
```

#### Core 1 Assembly program

```assembly
# Increment P0ToP1_0 every ~ second, repeating
addi  x13, x0, 0x2F                 # Approx 1 second delay for 12.5MHz clk

lui  x2,  0x20000
srli x2,  x2, 12               # x2 = 0x00020000 IO port base address 
addi x2,  x2, 8                # x2 = 0x00020008 P1ToP0_0 input port  

decrDelayCount:                # use long instruction loop to implement ~1 second delay
 addi  x13, x13,  -1           # decrement delayCount   
 bne   x13, x0, decrDelayCount # stay in loop if delayCount > 0 
 addi  x13, x0, 0x2F                # Approx 1 second delay for 12.5MHz clk
  
 lw   x5, 0(x2)                # load (read) 0x00020008 (P1ToP0_0) input to x5, core 0 to core 1
 beq  x0,  x0, decrDelayCount  # unconditional branch
```

In [33]:
Core0Program = [
  "02F00693", "20000137", "00C15113", "FFF68693", "FE069EE3", "02F00693", "00118193", "00312023",
  "FE0006E3", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000"]

Core1Program = [
  "02F00693", "20000137", "00C15113", "00810113", "FFF68693", "FE069EE3", "02F00693", "00012283", 
  "FE0008E3", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000"]

In [34]:
# Core 0
load_program(Core0Program, 0)
read_IM(0)
reset_sys(0)
activate_processor(0)

# Core 1
load_program(Core1Program, 1)
read_IM(1)
activate_processor(1)

Programming core 0...

Reading Core 0 IM...

['0x2f00693', '0x20000137', '0xc15113', '0xfff68693', '0xfe069ee3', '0x2f00693', '0x118193', '0x312023', '0xfe0006e3', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
Core 0 Activated!
Programming core 1...

Reading Core 1 IM...

['0x2f00693', '0x20000137', '0xc15113', '0x810113', '0xfff68693', '0xfe069ee3', '0x2f00693', '0x12283', '0xfe0008e3', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0

In [37]:
core0_reg_data = read_all_registers(0)
print("core 0 -> core 1 Data: " + str(read_P1ToP0_0(1)) + "\n") 
core1_reg_data = read_all_registers(1)

# print("\nP1 -> P0 Data: " + str(read_P1ToP0_0(1)) + "\n") 

print("Register Name\t|\tCore 0\t\t|\tCore 1 ")
print("----------------------------------------------------------------")
for i in range(0, 32):
    
    print("x" + str(i) + (" *" if (i == 3 or i == 5) else "") + "\t\t|\t" + core0_reg_data[i] + (" *" if (i == 3) else "") + "\t\t|\t" + core1_reg_data[i] + (" *" if (i == 5) else ""))

core 0 -> core 1 Data: 0x7

Register Name	|	Core 0		|	Core 1 
----------------------------------------------------------------
x0		|	0x0		|	0x0
x1		|	0x0		|	0x0
x2		|	0x20000		|	0x20008
x3 *		|	0x6 *		|	0x0
x4		|	0x0		|	0x0
x5 *		|	0x0		|	0x7 *
x6		|	0x0		|	0x0
x7		|	0x0		|	0x0
x8		|	0x0		|	0x0
x9		|	0x0		|	0x0
x10		|	0x0		|	0x0
x11		|	0x0		|	0x0
x12		|	0x0		|	0x0
x13		|	0x2a		|	0x18
x14		|	0x0		|	0x0
x15		|	0x0		|	0x0
x16		|	0x0		|	0x0
x17		|	0x0		|	0x0
x18		|	0x0		|	0x0
x19		|	0x0		|	0x0
x20		|	0x0		|	0x0
x21		|	0x0		|	0x0
x22		|	0x0		|	0x0
x23		|	0x0		|	0x0
x24		|	0x0		|	0x0
x25		|	0x0		|	0x0
x26		|	0x0		|	0x0
x27		|	0x0		|	0x0
x28		|	0x0		|	0x0
x29		|	0x0		|	0x0
x30		|	0x0		|	0x0
x31		|	0x0		|	0x0


### 8. Application 2

In core 0, x3 is incremented by 1 and the updated value is sent to core 1 while core 1 further increments the recieved value by 1 and sends it back to core 0

#### Core 0 program

```assembly
# Increment P0ToP1_0 every ~ second, repeating
addi  x13, x0, 0x2              
addi x3, x0, 0x48

lui  x1,  0x20000
srli x1,  x1, 12               # x2 = 0x00020000 process-processor port base address 

lui  x2,  0x20000
srli x2,  x2, 12               # x2 = 0x00020000 IO port base address 
addi x2,  x2, 8                # x2 = 0x00020008 P1ToP0_0 input port  

mainLoop:   
 addi x4, x3,  1                # increment x4
 sw   x4, 0(x1)                 # store(write) x3 to 0x00020000(P0ToP1_0), core 1 to core 0

 addi x12, x4, 1
 
 delayLoop:
 	addi  x13, x13,  -1           # decrement delayCount   
 	bne   x13, x0, delayLoop # stay in loop if delayCount > 0 
 addi  x13, x0, 0x2 
 
 lw   x3, 0(x2)                # Read data from core 1 via P1ToP0_0
 
 bne   x12, x3, mainLoop      # stay in loop until data recieved from core 1  

beq  x0,  x0, mainLoop   # unconditional branch
```

#### Core 1 program
```assembly
# Increment P0ToP1_0 every ~ second, repeating
addi  x13, x0, 0x2F                 # Approx 1 second delay for 12.5MHz clk

lui  x1,  0x20000
srli x1,  x1, 12               # x2 = 0x00020000 process-processor port base address 

lui  x2,  0x20000
srli x2,  x2, 12               # x2 = 0x00020000 IO port base address 
addi x2,  x2, 8                # x2 = 0x00020008 P1ToP0_0 input port  

decrDelayCount:                # use long instruction loop to implement ~1 second delay
 addi  x13, x13,  -1           # decrement delayCount   
 bne   x13, x0, decrDelayCount # stay in loop if delayCount > 0 
 addi  x13, x0, 0x2F                # Approx 1 second delay for 12.5MHz clk

 lw   x5, 0(x2)                # load (read) 0x00020008 (P1ToP0_0) input to x5, core 0 to core 1
 addi x6, x5, 1				   # Increment x5
 sw   x6, 0(x1)
 beq  x0,  x0, decrDelayCount  # unconditional branch
```

In [38]:
core0Program = [
  "00200693", "04800193", "200000B7", "00C0D093", "20000137", "00C15113", "00810113", "00118213",
  "0040A023","00120613", "FFF68693", "FE069EE3", "00200693", "00012183", "FE3612E3", "FE0000E3", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000"]

core1Program = [
  "02F00693", "200000B7", "00C0D093", "20000137", "00C15113", "00810113", "FFF68693", "FE069EE3",
  "02F00693", "00012283", "00128313", "0060A023", "FE0004E3", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", 
  "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000", "00000000"]

In [41]:
# Core 0
load_program(core0Program, 0)
read_IM(0)
reset_sys(0)


# Core 1
load_program(core1Program, 1)
read_IM(1)

activate_processor(0)
activate_processor(1)

Programming core 0...

Reading Core 0 IM...

['0x200693', '0x4800193', '0x200000b7', '0xc0d093', '0x20000137', '0xc15113', '0x810113', '0x118213', '0x40a023', '0x120613', '0xfff68693', '0xfe069ee3', '0x200693', '0x12183', '0xfe3612e3', '0xfe0000e3', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0']
Programming core 1...

Reading Core 1 IM...

['0x2f00693', '0x200000b7', '0xc0d093', '0x20000137', '0xc15113', '0x810113', '0xfff68693', '0xfe069ee3', '0x2f00693', '0x12283', '0x128313', '0x60a023', '0xfe0004e3', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0', '0x0'

In [46]:
# Core 0
load_program(core0Program, 0)
reset_sys(0)

# Core 1
load_program(core1Program, 1)

# Activate both cores
activate_processor(0)
activate_processor(1)

# Read Register and Ports
core0_P0ToP1_0_data = read_P0ToP1_0(0) 
core1_P1ToP0_0_data = read_P1ToP0_0(1)

core0_reg_data = read_all_registers(0)

core0_P1ToP0_0_data = read_P1ToP0_0(0)
core1_P0ToP1_0_data = read_P0ToP1_0(1)
core0_x3_dat = read_register("3", 0)
core1_reg_data = read_all_registers(1)


print("\t\t\t |Core 0\t\t\t\t\t|Core 1")
print("\t\t\t |----------------------------------------------|------------------------------------------")
print("Program\t\t\t |x3\tx4\tP0ToP1_0\tP1ToP0_0\t|x5\tx6\tP0ToP1_0\tP1ToP0_0")
print("-------------------------|----------------------------------------------|------------------------------------------")
print("c0 x4 = x3 + 1 \t\t |" + core0_reg_data[3] + "\t" + core0_reg_data[4] + "\t\t\t\t\t|")
print("c0 store x4 in P0ToP1_0  |\t\t" + str(core0_P0ToP1_0_data) + "\t\t\t\t|\t\t\t\t" + str(core1_P1ToP0_0_data))
print("c1 load x5 from P1ToP0_0 |\t\t\t\t\t\t|" + core1_reg_data[5])
print("c1 x6 = x5 + 1 \t\t |\t\t\t\t\t\t|\t" + core1_reg_data[6])
print("c1 store x6 in P0ToP1_0  |\t\t\t\t"+ str(core0_P1ToP0_0_data) + "\t\t|\t\t" + str(core1_P0ToP1_0_data))
print("c0 load x3 from P1ToP0_0 |" + core0_x3_dat + "\t\t\t\t\t\t|")

			 |Core 0					|Core 1
			 |----------------------------------------------|------------------------------------------
Program			 |x3	x4	P0ToP1_0	P1ToP0_0	|x5	x6	P0ToP1_0	P1ToP0_0
-------------------------|----------------------------------------------|------------------------------------------
c0 x4 = x3 + 1 		 |0x16	0x17					|
c0 store x4 in P0ToP1_0  |		0x17				|				0x17
c1 load x5 from P1ToP0_0 |						|0x17
c1 x6 = x5 + 1 		 |						|	0x18
c1 store x6 in P0ToP1_0  |				0x18		|		0x18
c0 load x3 from P1ToP0_0 |0x18						|
